In [1]:
import pandas as pd
import numpy as np
import random
import time

In [87]:
upbformstable = pd.read_csv('upbformstable.txt',sep='\t', skiprows=1, header=None, names=['roll','form'])
upbformstable

,roll,form
0,1,Normal human
1,2,Normal human
2,3,Normal human
3,4,Normal human
4,5,Normal human
...,...,...
95,96,Energy
96,97,Ethereal
97,98,Undead
98,99,Mutant - Breed


In [11]:
upb_form = upbformstable['form'].sample()
upb_form

38    Android
Name: form, dtype: object

In [89]:
upbnoofpowerstable = pd.read_csv('upbnoofpowers.txt',sep='\t', skiprows=1, header=None, names=['roll','powers_min','powers_max'])
upbnoofpowerstable

,roll,powers_min,powers_max
0,0,14,16
1,1,1,3
2,2,1,3
3,3,1,3
4,4,1,3
...,...,...,...
95,95,10,12
96,96,10,12
97,97,10,12
98,98,12,14


In [16]:
upb_powers = upbnoofpowerstable.sample()
upb_powers

,roll,powers_min,powers_max
28,28,3,5


In [90]:
upbpowercategories = pd.read_csv('upbpowercategories.txt',sep='\s+', header=None, names=['roll','category'] )
upbpowercategories

,roll,category
0,0,17000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
...,...,...
95,95,17000
96,96,17000
97,97,17000
98,98,17000


In [91]:
#upbpowerlist = pd.read_csv(r'J:\CLONE\WORK2\advnoofcontacts\upbpowerlist.txt', sep=' ', names=['roll','c','z','power'],header=None,engine="python")
upbpowerlist = pd.read_csv('upbpowerlist.txt', names=['test'], header=None,engine="python")
upbpowerlist

,test
0,1001 1 0 Body Armor
1,1002 1 0 Body Armor
2,1003 1 0 Body Armor
3,1004 1 0 Body Armor
4,1005 1 0 Body Armor
...,...
1695,17096 1 0 Water Walking
1696,17097 1 0 Water Walking
1697,17098 1 0 Whirlwind
1698,17099 1 0 Whirlwind


In [92]:
upbpowerlist[['roll','c','z','power']] = upbpowerlist["test"].str.split(" ", 3, expand=True)

upbpowerlist[['roll']] = upbpowerlist[['roll']].astype('int')
upbpowerlist[['c']] = upbpowerlist[['c']].astype('int')
upbpowerlist[['z']] = upbpowerlist[['z']].astype('int')

upbpowerlist.head(32)

,test,roll,c,z,power
0,1001 1 0 Body Armor,1001,1,0,Body Armor
1,1002 1 0 Body Armor,1002,1,0,Body Armor
2,1003 1 0 Body Armor,1003,1,0,Body Armor
3,1004 1 0 Body Armor,1004,1,0,Body Armor
4,1005 1 0 Body Armor,1005,1,0,Body Armor
5,1006 1 0 Body Armor,1006,1,0,Body Armor
6,1007 1 0 Body Armor,1007,1,0,Body Armor
7,1008 1 0 Body Armor,1008,1,0,Body Armor
8,1009 1 0 Body Armor,1009,1,0,Body Armor
9,1010 1 0 Body Armor,1010,1,0,Body Armor


In [76]:
#get the list where >= the number and <= number + 1000
power_categories = []
powers_list = []

for p in range(upb_powers['powers_min'].values[0]):
#upb_upbpowercategories = upbpowercategories.sample()  #need one for each power
#upbpowercategories
    print(p)
    c = upbpowercategories.sample()
    power_categories.append(c['category'].values[0])
    #power_subset = upbpowerlist.loc[(upbpowerlist['roll'] >= c['category'].values[0])]        
    power_subset = upbpowerlist.loc[(upbpowerlist['roll'] >= c['category'].values[0]) & (upbpowerlist['roll'] < (c['category'].values[0] + 1000))]    
    #print(power_subset.shape)
    powers_list.append(power_subset.sample())

0
1
2


In [77]:
print(power_categories)
print(powers_list)

[1000, 10000, 13000]
[                  test  roll  c  z       power
5  1006 1 0 Body Armor  1006  1  0  Body Armor,                           test   roll  c  z            power
940  10041 2 0 Disintegration*  10041  2  0  Disintegration*,                           test   roll  c  z           power
1253  13054 1 0 Hyper-Strength  13054  1  0  Hyper-Strength]


In [63]:
upbpowercategories.dtypes

roll        int64
category    int64
dtype: object

In [71]:
upbpowerlist.dtypes

test     object
roll      int32
c         int32
z         int32
power    object
dtype: object

In [93]:
column_roll = pd.read_csv('formabilitycolumns.csv')
column_use = column_roll.loc[column_roll['Type'] == upb_form.values[0]]['Column'].values[0]
column_use

4

In [97]:
#want to test the forms
FASERIP = {"F":"Ty","A":"Ty","S":"Ty","E":"Ty","R":"Ty","I":"Ty","P":"Ty"}
for ability in FASERIP.keys():
    FASERIP[ability] = column_roll[str(column_use)].sample()
    
FASERIP    

{'F': 14    Ty
 Name: 4, dtype: object,
 'A': 61    Rm
 Name: 4, dtype: object,
 'S': 19    Gd
 Name: 4, dtype: object,
 'E': 37    Gd
 Name: 4, dtype: object,
 'R': 54    Rm
 Name: 4, dtype: object,
 'I': 5    Pr
 Name: 4, dtype: object,
 'P': 44    Ex
 Name: 4, dtype: object}